- To run ETL pipeline that cleans data and stores in database
        `python data/process_data.py data/disaster_messages.csv data/disaster_categories.csv data/DisasterResponse.db`
- To run ML pipeline that trains classifier and saves
        `python models/train_classifier.py data/DisasterResponse.db models/classifier.pkl`

# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
import os
import nltk
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
import re
import sys
import string
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import seaborn as sns
from numpy import argmax
from sklearn.metrics import multilabel_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
import pickle
import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hamza\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\hamza\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hamza\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [59]:
def load_data(file_name):
    path = os.getcwd() +'//data//' + file_name + '.db'
    engine = create_engine('sqlite:///' + path)
    df = pd.read_sql_table(file_name, engine)
    return df

In [60]:
df = load_data('DisasterResponse')

In [61]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [62]:
categories = df.iloc[:, 4:].columns

In [63]:
categories

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(df.message, df.iloc[:,4:], train_size=0.85) 

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, "", text).lower().split()
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in text]
    tokens = word_tokenize(' '.join(stripped))
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)

    return clean_tokens
    #return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
y_pred = pipeline.predict(X_valid)
y_test = np.array(y_valid)

In [ ]:
print(classification_report(np.concatenate(y_test), np.concatenate(y_pred)))

In [ ]:
def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):

    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )

    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_xlabel('True label')
    axes.set_ylabel('Predicted label')
    axes.set_title("Class - " + class_label)
    
confusion_mat = multilabel_confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(9, 4, figsize=(20, 20))
    
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_mat, categories):
    print_confusion_matrix(cfs_matrix, axes, label, ["Y", "N"])

fig.tight_layout()
plt.show() 

In [ ]:
for matrix in tuple(zip(categories, confusion_mat)):
    print(matrix)

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipe2=Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(RandomForestClassifier())),
               ])

In [ ]:
parameters = {
#    'tfidf__use_idf': (True, False), 
#     'clf__estimator__n_estimators': [50, 100],
#     'clf__estimator__min_samples_split': [2, 3, 4],
     'vect__max_features': (None, 5000, 10000),
     'vect__max_df': (0.5, 0.75, 1.0),
     'vect__ngram_range': ((1, 1), (1, 2)),
    }

In [ ]:
cv = GridSearchCV(pipe2, cv=2, param_grid=parameters)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_valid)

In [ ]:
y_test = np.array(y_valid)
x = classification_report(np.concatenate(y_test), np.concatenate(y_pred))
print(x, y_pred, y_test)

In [ ]:
best_parameters = cv.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
parameters = {
#    'tfidf__use_idf': (True, False), 
#     'clf__estimator__n_estimators': [50, 100],
#     'clf__estimator__min_samples_split': [2, 3, 4],
    'vect__max_df': [0.5],
    'vect__max_features': [5000],
    'vect__ngram_range': [(1, 2)],
    }

In [ ]:
grid = GridSearchCV(pipe2, cv=2, param_grid=parameters)

In [ ]:
grid.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_valid)
y_test = np.array(y_valid)
x = classification_report(np.concatenate(y_test), np.concatenate(y_pred))
print(x)

In [ ]:
confusion_mat = multilabel_confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(9, 4, figsize=(20, 20))
    
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_mat, categories):
    print_confusion_matrix(cfs_matrix, axes, label, ["Y", "N"])

fig.tight_layout()
plt.show() 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipe2=Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                ('tfidf', TfidfTransformer()),
                ('svd', TruncatedSVD()),
                ('clf', MultiOutputClassifier(RandomForestClassifier())),
               ])

In [ ]:
from sklearn.pipeline import Pipeline, FeatureUnion


In [ ]:
pipe2.fit(X_train, y_train)

In [ ]:
y_pred = pipe2.predict(X_valid)
y_test = np.array(y_valid)

In [ ]:
x = classification_report(np.concatenate(y_test), np.concatenate(y_pred))
print(x, y_pred, y_test)

In [ ]:
confusion_mat = multilabel_confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(9, 4, figsize=(20, 20))
    
for axes, cfs_matrix, label in zip(ax.flatten(), confusion_mat, categories):
    print_confusion_matrix(cfs_matrix, axes, label, ["Y", "N"])

fig.tight_layout()
plt.show()

### 9. Export your model as a pickle file

In [ ]:
classifier = open(model_filepath, 'wb')
pickle.dump(model, classifier)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [5]:
def load_data(database_filepath):
    engine = create_engine('sqlite:///' + database_filepath)
    df = pd.read_sql_table('DisasterResponse', engine)
    categories = df.iloc[:, 4:].columns
    X, Y = df.message, df.iloc[:,4:]
    return X, Y, categories


def tokenize(text):
    url_regex = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(url_regex, "", text).lower().split()
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in text]
    tokens = word_tokenize(' '.join(stripped))
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok)
        clean_tokens.append(clean_tok)

    return clean_tokens

def build_model():
    pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('svd', TruncatedSVD()),
    ('clf', MultiOutputClassifier(RandomForestClassifier())),
])

    parameters = {#best parameters estimated using gridsearch
    #    'tfidf__use_idf': (True, False), 
    #     'clf__estimator__n_estimators': [50, 100],
    #     'clf__estimator__min_samples_split': [2, 3, 4],
    'vect__max_df': [0.5],
    'vect__max_features': [5000],
    'vect__ngram_range': [(1, 2)],
    }
    grid = GridSearchCV(pipeline, cv=2, param_grid=parameters)
    return grid

def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):

    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )
    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_xlabel('True label')
    axes.set_ylabel('Predicted label')
    axes.set_title("Class - " + class_label)



def evaluate_model(model, X_test, Y_test, category_names):
    y_pred = model.predict(X_test)
    y_test = np.array(Y_test)

    x = classification_report(np.concatenate(np.array(Y_test)), np.concatenate(y_pred))
    print('Classification report: ',x)

    confusion_mat = multilabel_confusion_matrix(y_test, y_pred)
    print(confusion_mat)
    fig, ax = plt.subplots(7, 5, figsize=(20, 20))    
    for axes, cfs_matrix, label in zip(ax.flatten(), confusion_mat, categories):
        print_confusion_matrix(cfs_matrix, axes, label, ["Y", "N"])

    fig.tight_layout()
    plt.show()
    



def save_model(model, model_filepath):
    # Open the file to save as pkl file
    classifier = open(model_filepath, 'wb')
    pickle.dump(model, classifier)
    # Close the pickle instances
    classifier.close()

In [6]:
database_filepath = 'data/DisasterResponse.db'
model_filepath = 'models/classifier.pkl'

In [7]:
X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
Y_train.head()

In [ ]:
print('Building model...')
model = build_model()

print('Training model...')
model.fit(X_train, Y_train)

In [ ]:
print('Evaluating model...')
evaluate_model(model, X_test, Y_test, category_names)

In [ ]:
print('Saving model...\n    MODEL: {}'.format(model_filepath))
save_model(model, model_filepath)

print('Trained model saved!')

# Testing

In [30]:
X, Y, category_names = load_data(database_filepath)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state = 42)

In [97]:
X_test.iloc[5]

'To ensure trouble-free passage of flood water and prevent flooding of houses, all the necessary organizational and practical measures are being taken," the press service of the Oblast\'s Akimat said. The program requires that the reconstruction work should stick to the conbination of the reconstrctuion of urban housing and the development of urbanization; the reconstruction of rural housing and the construction of socialism news countries and the poverty-relief; the governmental management and the marketing exercise; the housing strengthening and reparing and the housing building. Workers are also preparing to purchase large quantities of tin in order to repair some of the hundreds of homes damaged by the storms. Women are able to come to the riverbed, scoop into the sand, and find water rather than walking for hours in search of the life sustaining resource. In the Labutta Township, ADRA is partnering with various donors including the United States Agency for International Developmen

In [31]:
model = joblib.load("models\classifier.pkl")

In [51]:
predictions = model.predict([X_test.iloc[5]])

In [53]:
y_pred = predictions[0]
y_test = np.array(Y_test.iloc[5])

In [88]:
true_labels = {categories[i] for i,prob in enumerate(y_test) if prob == 1}
predicted_labels = {categories[i] for i,prob in enumerate(y_pred) if prob == 1}
x = true_labels.intersection(predicted_labels)
(len(x) / len(true_labels)) * 100